In [ ]:
import pathlib
import os
import numpy as np
import pandas as pd
from ase.visualize import view
from ase.build.tools import sort as ase_sort
from ase.atoms import Atoms

from aiida import orm, load_profile
from aiida.orm import load_node
from aiida.engine import submit

from qe_tools import CONSTANTS

from aiida_quantumespresso_hp.workflows.hubbard import SelfConsistentHubbardWorkChain
from aiida_quantumespresso.data.hubbard_structure import HubbardStructureData

from project_settings import *

load_profile()

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
fully_lithiated_df = pd.read_pickle(os.path.join(project_dir, 'data', 'fully_lithiated_df.pkl'))
mn_olivine_ase = fully_lithiated_df['ase_in'].values[0]

sorting_dict = {
    'Mn': 0,
    'O': 1,
    'P': 2,
    'Li': 3,
}

# ! Does not take into account initial magmoms for now
mn_olivine_ase_ordered = Atoms(sorted(mn_olivine_ase, key=lambda x: sorting_dict[x.symbol]))
mn_olivine_ase_ordered.set_cell(mn_olivine_ase.get_cell())

mn_olivine_aiida = orm.StructureData(ase=mn_olivine_ase_ordered)

## HubbardStructureData initialization
Let's initialize the HubbardStructureData with the olivine structure!

In [ ]:
hubbard_data_inv = HubbardStructureData(structure=mn_olivine_aiida)

In [ ]:
# hubbard_data.reorder_atoms()

## Initializing the on-site Hubbard
Let's initialize the on-site Hubbard parameter for the titanium atom.

In [ ]:
# Taken from parameters.in of:
# /home/jgeiger/projects/bat_uv_ml/data/olivine_iurii/LixMnPO4/Li1.00/DFT_plus_UV/9_PDOS/LMPO.scf.1.in
hubbard_data_inv.initialize_onsites_hubbard('Mn', '3d', 4.5618)

Here how it is stored in the class.

In [ ]:
hubbard_data_inv.hubbard_parameters

## Initializing the inter-site Hubbard
Let's initialize the inter-site Hubbard parameter for the titanium and oxygen atoms.

In [ ]:
hubbard_data_inv.initialize_intersites_hubbard('Mn', '3d', 'O', '2p', 0.0001, number_of_neighbours=7) 
# ! Ti has 6 oxygen neighbors in BaTiO3. Similarly, Mn has 6 oxygen neighbors in LMPO (olivine). So, we need to use N+1 in the call to initialize_intersites_hubbard, as the value is used for list slicing, like [:number_of_neighbours], which is exclusive of the last index.

The parameters are saved in the property `hubbard_parameters` as a list.

In [ ]:
hubbard_data_inv.hubbard_parameters

In [ ]:
print(hubbard_data_inv.get_quantum_espresso_hubbard_card())
print(hubbard_data_inv.get_quantum_espresso_hubbard_parameters())

In [ ]:
print(type(mn_olivine_aiida))
print(mn_olivine_aiida.get_ase().get_chemical_symbols())
from aiida.orm.nodes.data.structure import StructureData

In [ ]:
hubbard_data_inv.store()
hubbard_data_inv.pk

In [ ]:
inverted_hubbard_data_pk = 3015

## Running the `SelfConsistentHubbardWorkChain`

In [ ]:
hubbard_data_inv = orm.load_node(inverted_hubbard_data_pk)  # I load the node from the database instead of always regenerating it
hubbard_data_inv.hubbard_parameters
# print(hubbard_data.get_quantum_espresso_hubbard_card())
# type(hubbard_data)

# ? On-site gets only stored once, if I am not mistaken.
# ? Index is changed from 0-based (Python) to 1-based (Quantum ESPRESSO)

In [ ]:
pw_code = orm.load_code('qe-dev-pw@lumi-small')
hp_code = orm.load_code('qe-dev-hp@lumi-small')

builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data_inv,
    protocol='fast',
    overrides=pathlib.Path('hubbard_overrides.yaml')
)

builder.skip_first_relax = True

In [ ]:
olivine_workchain_submit_inv = submit(builder)
print(olivine_workchain_submit_inv)

In [ ]:
olivine_workchain_submit_inv_pk_first = 2914
olivine_workchain_submit_inv_pk_second = 2978
olivine_workchain_submit_inv_pk_second = 3040

In [ ]:
!verdi process status 3040
!verdi process report 3040
!verdi process status 3071

In [ ]:
!verdi process status 3071
!verdi process report 3071
!verdi process show 3071

In [ ]:
%%bash
# verdi process show 2914
# printf '=%.0s' {1..100}
# echo
# verdi process status 2914
# printf '=%.0s' {1..100}
# echo
verdi process status 2914
printf '=%.0s' {1..100}
echo

In [ ]:
!verdi process show 2943

In [ ]:
%%bash
# Previous erroneous workchain, where Hubbard atoms were not in the beginning,
# which causes a problem in the Hubbard code.
verdi process show 2860
printf '=%.0s' {1..100}
echo
verdi process status 2860
printf '=%.0s' {1..100}
echo
verdi process report 2860
printf '=%.0s' {1..100}
echo

In [ ]:
import importlib
from aiida_quantumespresso_hp.calculations.functions import structure_relabel_kinds
importlib.reload(structure_relabel_kinds)


hubbard_data = orm.load_node(2889)  # I load the node from the database instead of always regenerating it
failed_hp_workchain = load_node(2945)

qe_hubbard_card_before = hubbard_data.get_quantum_espresso_hubbard_card()
# print('Hubbard card before structure_relabel_kinds\n\n', qe_hubbard_card_before)

relabel_output = structure_relabel_kinds.structure_relabel_kinds(
    hubbard_structure=failed_hp_workchain.outputs.hubbard_structure,
    hubbard=failed_hp_workchain.outputs.hubbard
)

qe_hubbard_card_after = relabel_output['hubbard_structure'].get_quantum_espresso_hubbard_card()

print('Hubbard card after structure_relabel_kinds\n\n', qe_hubbard_card_after)